<a href="https://colab.research.google.com/github/GGMmattos/PIC/blob/main/TOLD_BR_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing needed libraries
!pip install -U scikit-learn
!pip install -U spacy
!python -m spacy download pt


!pip install wandb
!pip install --upgrade simpletransformers
!pip install --upgrade transformers

from pandas.core.window.ewm import OnlineExponentialMovingWindow
import pandas as pd #Biblioteca para carregamento de arquivo csv
import numpy as np
import seaborn as sns #Para visualização de gráficos
import matplotlib.pyplot as plt #Para visualização de gráficos
import plotly.express as px #Biblioteca para geração de gráfico dinámico
%matplotlib inline
from collections import Counter


#NLTK tools for text processing

import re, nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


#Modeling packages

from sklearn.model_selection import train_test_split
from sklearn.metrics._plot.confusion_matrix import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity ='all'

from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression



from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
import numpy as np
import sklearn
import pickle
import spacy
from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads
from transformers import pipeline
from sklearn.model_selection import train_test_split


BERT IMBAU - Sem pré-processamento

In [ ]:
base_hatebr = pd.read_csv('/content/drive/MyDrive/PIC /Dataset/HateBR.csv')


In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


told_br = pd.read_csv("/content/drive/MyDrive/PIC /Dataset/told_br_binary.csv")

# Preparing train data
Told_br_train_data = told_br.iloc[:,[1, 24]]
train,eva = train_test_split(Told_br_train_data,test_size = 0.2)

train_df = pd.DataFrame({
    'text': train['text'],
    'label': train['toxic_1']
})

eval_df = pd.DataFrame({
    'text': eva['text'],
    'label': eva['toxic_1']
})

#Optional model configuration
model_args = ClassificationArgs(num_train_epochs=1)

#Create a ClassificationModel
model = ClassificationModel(
    "bert", "neuralmind/bert-base-portuguese-cased", args=model_args,
    use_cuda=False
)

model.train_model(train_df)


Verificando accuracy

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)
lst = []
for arr in model_outputs:
    lst.append(np.argmax(arr))
true = eval_df['label'].tolist()
predicted = lst
sklearn.metrics.accuracy_score(true,predicted)

BERT IMBAU - com pré-processamento Bert-Base

In [ ]:
told_br = pd.read_csv("/content/drive/MyDrive/PIC /Dataset/told_br_binary.csv")

In [ ]:
npl = spacy.load("pt_core_news_sm")
def preprocess(text):
  comentarios = text.lower()
  # only_latters =re.sub("[^a-zA-Z]", " ", comentarios) # somente letras
  doc = npl(comentarios)
  filtered_result = []
  for token in doc:
    if token.is_stop or token.is_punct :
      continue
    filtered_result.append(token.lemma_)
  return " ".join(filtered_result)

# print(preprocess(base_hatebr['instagram_comments']))
told_br['preprocessed_comments'] =  told_br['text'].apply(preprocess)

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Preparing train data
Told_br_train_data = told_br.iloc[:,[33,24]]
train,eva = train_test_split(Told_br_train_data,test_size = 0.2)

train_df = pd.DataFrame({
    'text': train['preprocessed_comments'],
    'label': train['toxic_1']
})

eval_df = pd.DataFrame({
    'text': eva['preprocessed_comments'],
    'label': eva['toxic_1']
})

#Optional model configuration
model_args = ClassificationArgs(num_train_epochs=1)

#Create a ClassificationModel
model = ClassificationModel(
    "bert", "neuralmind/bert-base-portuguese-cased", args=model_args,
    use_cuda=False
)

model.train_model(train_df)

result, model_outputs, wrong_predictions = model.eval_model(eval_df)


In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)
lst = []
for arr in model_outputs:
    lst.append(np.argmax(arr))
true = eval_df['label'].tolist()
predicted = lst
sklearn.metrics.accuracy_score(true,predicted)
